In [ ]:
import pandas as pd
import plotly.express as px

In [ ]:
df = pd.read_pickle("results.pkl")

In [ ]:
df["Average Add Time"] = df["Add Times"].map(lambda x: sum(x) / len(x))
df["Average Perturb Time"] = df["Perturb Times"].map(lambda x: sum(x) / len(x))
df.sort_values(["experiment_type", "Trial Number"])
df["nodes"] = df["nodes"].astype(str)

In [ ]:
df.to_csv("results.csv")

In [ ]:
df.columns

In [ ]:
df.head()
a = df[df["experiment_type"]=="Single"]
a

In [ ]:
px.bar(a, x="nodes", y="Time Taken", barmode="group", color="Path Selector", facet_col="top_k", facet_row="k", pattern_shape="Status", hover_data=["Average Add Time", "Average Perturb Time", "Original Distance", "Number of Paths", "Number of Edges", "Iterations"])